# Euromillions

In [ ]:
# Import des librairies
import pandas as pd
import csv
import os
from IPython.display import display
import requests
import zipfile
import io
from pathlib import Path

# Variables utiles
dir_raw = Path(r"C:\Users\hbern\Downloads\notebooks\euromillions-git\data\raw")
dir_processed = Path(r"C:\Users\hbern\Downloads\notebooks\euromillions-git\data\processed")
fdj_url = "https://www.sto.api.fdj.fr/anonymous/service-draw-info/v3/documentations"

## Import des fichiers

In [ ]:
# Liste des fichiers
files = {
    "euromillions": "1a2b3c4d-9876-4562-b3fc-2c963f66afa8",
    "euromillions_2": "1a2b3c4d-9876-4562-b3fc-2c963f66afa9",
    "euromillions_3": "1a2b3c4d-9876-4562-b3fc-2c963f66afb6",
    "euromillions_4": "1a2b3c4d-9876-4562-b3fc-2c963f66afc6", 
    "euromillions_201902": "1a2b3c4d-9876-4562-b3fc-2c963f66afd6",  
    "euromillions_202002": "1a2b3c4d-9876-4562-b3fc-2c963f66afe6"
}

def download_and_unzip(file_name: str, uuid: str, fdj_url: str, dir_raw: str | Path):
    dir_raw = Path(dir_raw)
    dir_raw.mkdir(parents=True, exist_ok=True)

    url = f"{fdj_url}/{uuid}"
    print(f"📥 Téléchargement de {url}")

    r = requests.get(url)
    r.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        # on prend le premier CSV trouvé dans le ZIP
        csv_names = [n for n in z.namelist() if n.lower().endswith(".csv")]
        if not csv_names:
            raise ValueError(f"Aucun CSV dans l'archive {uuid}")

        interne = csv_names[0]
        z.extract(interne, dir_raw)

    source = dir_raw / interne
    raw = dir_raw / f"{file_name}.csv"

    source.rename(raw)
    print(f"✅ {raw.name} créé")

for file_name, uuid in files.items():
    fp = dir_raw / f"{file_name}.csv"

    # cas spécial : toujours réimporter le dernier fichier
    if file_name == "euromillions_202002":
        print(f"♻️ Réimport forcé de {fp.name}")
        download_and_unzip(file_name, uuid, fdj_url, dir_raw)
        continue

    # comportement normal pour les autres
    if fp.exists():
        print(f"⏭️ {fp.name} existe déjà")
        continue

    print(f"📂 {fp.name} absent → import")
    download_and_unzip(file_name, uuid, fdj_url, dir_raw)

## Fonction de lecture des fichiers 

In [ ]:
def read_csv(file: str, dir_raw: str) -> pd.DataFrame:
    """Lit un fichier CSV et renvoie un DataFrame pandas."""

    dir_raw = Path(dir_raw)

    file_path = dir_raw / file  
    
    encoding_type = "utf-8-sig"

    if file in ("euromillions_4.csv", "euromillions_201902.csv"):
        encoding_type = "cp1252"
    
    df = pd.read_csv(
        file_path,
        index_col=False,
        engine="python",
        sep=None,
        encoding=encoding_type,
        quotechar='"',
        escapechar='\\',
        skip_blank_lines=True,
        on_bad_lines="skip"
    )    

    # Remet les dates de tirage par ordre croissant, sauf le fichier euromillions_4 où elles le sont déjà
    if file != "euromillions_4.csv":
        df = df.iloc[::-1].reset_index(drop=True)
    
    return df

## Colonnes

In [ ]:
# Liste des colonnes à supprimer dans les fichiers

# fichier 1
cols_to_drop = ['date_de_forclusion', 'nombre_de_gagnant_au_rang2_en_france', 'nombre_de_gagnant_au_rang2_en_europe', 'rapport_du_rang2',
       'nombre_de_gagnant_au_rang3_en_france', 'nombre_de_gagnant_au_rang3_en_europe', 'rapport_du_rang3', 'nombre_de_gagnant_au_rang4_en_france', 'nombre_de_gagnant_au_rang4_en_europe', 'rapport_du_rang4',
       'nombre_de_gagnant_au_rang5_en_france', 'nombre_de_gagnant_au_rang5_en_europe', 'rapport_du_rang5', 'nombre_de_gagnant_au_rang6_en_france', 'nombre_de_gagnant_au_rang6_en_europe', 'rapport_du_rang6',
       'nombre_de_gagnant_au_rang7_en_france', 'nombre_de_gagnant_au_rang7_en_europe', 'rapport_du_rang7', 'nombre_de_gagnant_au_rang8_en_france', 'nombre_de_gagnant_au_rang8_en_europe', 'rapport_du_rang8',
       'nombre_de_gagnant_au_rang9_en_france', 'nombre_de_gagnant_au_rang9_en_europe', 'rapport_du_rang9', 'nombre_de_gagnant_au_rang10_en_france', 'nombre_de_gagnant_au_rang10_en_europe', 'rapport_du_rang10',
       'nombre_de_gagnant_au_rang11_en_france', 'nombre_de_gagnant_au_rang11_en_europe', 'rapport_du_rang11', 'nombre_de_gagnant_au_rang12_en_france', 'nombre_de_gagnant_au_rang12_en_europe', 'rapport_du_rang12',
       'numero_jokerplus', 'devise', 'Unnamed: 51']

# fichier 2
cols_to_drop += ['nombre_de_gagnant_au_rang13_en_france', 'nombre_de_gagnant_au_rang13_en_europe', 'rapport_du_rang13', 'Unnamed: 54']

# fichier 3
cols_to_drop += ['numero_My_Million']

# fichier 4
cols_to_drop += ['numéro_de_tirage_dans_le_cycle', 'nombre_de_gagnant_au_rang2_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang2_Euro_Millions_en_europe', 'rapport_du_rang2_Euro_Millions', 'nombre_de_gagnant_au_rang3_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang3_Euro_Millions_en_europe', 'rapport_du_rang3_Euro_Millions', 'nombre_de_gagnant_au_rang4_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang4_Euro_Millions_en_europe', 'rapport_du_rang4_Euro_Millions', 'nombre_de_gagnant_au_rang5_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang5_Euro_Millions_en_europe', 'rapport_du_rang5_Euro_Millions', 'nombre_de_gagnant_au_rang6_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang6_Euro_Millions_en_europe', 'rapport_du_rang6_Euro_Millions', 'nombre_de_gagnant_au_rang7_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang7_Euro_Millions_en_europe', 'rapport_du_rang7_Euro_Millions', 'nombre_de_gagnant_au_rang8_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang8_Euro_Millions_en_europe', 'rapport_du_rang8_Euro_Millions', 'nombre_de_gagnant_au_rang9_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang9_Euro_Millions_en_europe', 'rapport_du_rang9_Euro_Millions', 'nombre_de_gagnant_au_rang10_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang10_Euro_Millions_en_europe', 'rapport_du_rang10_Euro_Millions', 'nombre_de_gagnant_au_rang11_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang11_Euro_Millions_en_europe', 'rapport_du_rang11_Euro_Millions', 'nombre_de_gagnant_au_rang12_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang12_Euro_Millions_en_europe', 'rapport_du_rang12_Euro_Millions', 'nombre_de_gagnant_au_rang13_Euro_Millions_en_france', 'nombre_de_gagnant_au_rang13_Euro_Millions_en_europe', 'rapport_du_rang13_Euro_Millions', 'nombre_de_gagnant_au_rang1_Etoile+', 'rapport_du_rang1_Etoile+', 'nombre_de_gagnant_au_rang2_Etoile+', 'rapport_du_rang2_Etoile+', 'nombre_de_gagnant_au_rang3_Etoile+', 'rapport_du_rang3_Etoile+', 'nombre_de_gagnant_au_rang4_Etoile+', 'rapport_du_rang4_Etoile+',
       'nombre_de_gagnant_au_rang5_Etoile+', 'rapport_du_rang5_Etoile+', 'nombre_de_gagnant_au_rang6_Etoile+', 'rapport_du_rang6_Etoile+', 'nombre_de_gagnant_au_rang7_Etoile+', 'rapport_du_rang7_Etoile+', 'nombre_de_gagnant_au_rang8_Etoile+', 'rapport_du_rang8_Etoile+', 'nombre_de_gagnant_au_rang9_Etoile+', 'rapport_du_rang9_Etoile+', 'nombre_de_gagnant_au_rang10_Etoile+', 'rapport_du_rang10_Etoile+', 'numero_Tirage_Exceptionnel_Euro_Millions']

# fichier 201902
cols_to_drop += ['Unnamed: 75']

# fichier 202002
cols_to_drop += ['numero_Tirage_Exceptionnel_Euro_Million']

# print(cols_to_drop)
# print("Nombre de colonnes à supprimer : " + str(len(cols_to_drop)))
# Dédoublonne pour être sûr 
# cols_to_drop = list(set(cols_to_drop))
# print("Nombre de colonnes à supprimer (dédoublonnée) : " + str(len(cols_to_drop)))

## 1. Fichier euromillions.csv

- 13/02/2004 au 06/05/2011
- Tirages 1 à 378 (indices 0 à 377)

In [ ]:
df1 = read_csv("euromillions.csv", dir_raw)

# Supprime les colonnes inutiles 
df1 = df1.drop(columns=cols_to_drop, errors="ignore")

df1.head(df1.shape[0]) # 378,15

## 2. Fichier euromillions_2.csv

- 10/05/2011 au 31/01/2014
- Tirages 379 à 664

In [ ]:
df2 = read_csv("euromillions_2.csv", dir_raw)

# Supprime les colonnes inutiles 
df2 = df2.drop(columns=cols_to_drop, errors="ignore")

df2.head(df2.shape[0]) # (286, 15)

## 3. Fichier euromillions_3.csv

- 04/02/2014 au 23/09/2016
- Tirages 665 à 940
- Introduction numéro My Million

In [ ]:
df3 = read_csv("euromillions_3.csv", dir_raw)

# Supprime les colonnes inutiles 
df3 = df3.drop(columns=cols_to_drop, errors="ignore")

# Corrige en 23/09/2016 la date de la dernière ligne :
# 275 	2016077 	VENDREDI 	23/09/16 	
df3.loc[275, "date_de_tirage"] = "23/09/2016"

df3.head(df3.shape[0]) # (276, 15)

## 4. Fichier euromillions_4.csv

- 27/09/2016 au 26/02/2019
- Tirages 941 à 1290 
- Introduction Etoile+

In [ ]:
df4 = read_csv("euromillions_4.csv", dir_raw)

# Supprime les colonnes inutiles 
df4 = df4.drop(columns=cols_to_drop, errors="ignore")

df4.head(df4.shape[0]) # (253, 15)

## 5. Fichier euromillions_201902.csv

- 01/03/2019 au 31/01/2020
- Tirages 1194 à 1290  

In [ ]:
df5 = read_csv("euromillions_201902.csv", dir_raw)

# Supprime les colonnes inutiles 
df5 = df5.drop(columns=cols_to_drop, errors="ignore")

df5.head(df5.shape[0]) # (97, 15)

## 6. Fichier euromillions_202002.csv

- Depuis le 04/02/2020
- Depuis tirage 1291   

In [ ]:
def download_and_unzip(url: str, dir_raw: str | Path):
    """
        Télécharge un fichier ZIP depuis une URL et le décompresse dans le dossier indiqué.
    """
    dir_raw = Path(dir_raw)
    dir_raw.mkdir(parents=True, exist_ok=True)  # crée le dossier si besoin

    print(f"📥 Téléchargement de {url} ...")
    r = requests.get(url)
    r.raise_for_status()  # lève une erreur si le téléchargement échoue

    print("🗜️ Décompression...")
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        z.extractall(dir_raw)

    print(f"✅ Fichiers extraits dans : {dir_raw.resolve()}")

download_and_unzip("https://www.sto.api.fdj.fr/anonymous/service-draw-info/v3/documentations/1a2b3c4d-9876-4562-b3fc-2c963f66afe6", dir_raw)

df6 = read_csv("euromillions_202002.csv", dir_raw)

# Supprime les colonnes inutiles 
df6 = df6.drop(columns=cols_to_drop, errors="ignore")

df6.head(df6.shape[0]) # (XXX, 15)

print(f"Date du dernier tirage : {df6["date_de_tirage"].iloc[-1]}")

In [ ]:
colonnes_nouvelles = ["annee_numero_tirage", "jour_tirage", "date_tirage", 
               "boule_1", "boule_2", "boule_3", "boule_4", "boule_5",
               "etoile_1", "etoile_2",
               "boules_croissant", "etoiles_croissant",
               "nb_gagnants_r1_fr", "nb_gagnants_r1_eu", "rapport_r1"
              ]

# Liste des DataFrames
dataframes = [df1, df2, df3, df4, df5, df6]

# Boucle sur la liste
for df in dataframes:
    df.columns = colonnes_nouvelles

In [ ]:
# Concaténation
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

df.head(df.shape[0]) 

In [ ]:
# Etat du dataframe
df.dtypes
df.info()

df.isna()
df.isna().sum()

In [ ]:
# jour tirage = 'V' ou 'M' 
df["jour_tirage"] = df["jour_tirage"].astype(str).str[0]

# supprime les '-' en début et fin 
df["boules_croissant"] = df["boules_croissant"].astype(str).str.strip("-")
df["etoiles_croissant"] = df["etoiles_croissant"].astype(str).str.strip("-")

df.head(df.shape[0]) # 1893 rows × 15 columns

# Démarre l'index à 1 au lieu de 0 = correspond ainsi au numéro de tirage depuis le début
df = df.reset_index(drop=True)
df.index += 1

# Conversion de la colonne 'date' en datetime
df["date_tirage"] = pd.to_datetime(df["date_tirage"], dayfirst=True, errors="coerce")

In [ ]:
# Export du dataframe global
global_file_path = os.path.join(dir_processed, "global.csv")

# on supprime le fichier 'csv/global.csv' sil existe 
if os.path.exists(global_file_path):
    os.remove(global_file_path)

df_export = df.copy()
df_export["date_tirage"] = df_export["date_tirage"].dt.strftime("%d/%m/%Y")

df_export.to_csv(global_file_path, index=False, encoding="utf-8-sig")

## Nettoyage et processing

In [ ]:
# -----------------------------------------------------------------------
# 1) nettoyage
# -----------------------------------------------------------------------

# Normalisation des types
# - Conversion des colonnes numériques mal typées
for col in ["etoile_2", "rapport_r1"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Conversion de la date
if "date_tirage" in df.columns:
    df["date_tirage"] = pd.to_datetime(df["date_tirage"], errors="coerce", format="%Y%m%d").fillna(
        pd.to_datetime(df["date_tirage"], errors="coerce")
    )

## Affichage final

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    df["date_tirage"] = df["date_tirage"].dt.strftime("%d/%m/%Y")
    display(df)

In [ ]:
# display(df)